<a href="https://colab.research.google.com/github/nrkfeller/YCBS258/blob/master/DSDT_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q pyyaml

In [0]:
# Install dependencies
!apt install graphviz
!pip install pydot pydot-ng
!echo "Double check with Python 3"
!python -c "import pydot"

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, Flatten

print(tf.VERSION)
print(tf.keras.__version__)

## Models types
### Sequential
Simple stack of layers
### Functional
Multi input, multi output, shared layers, non sequential flows
### Model Subclassing
Customizable, define foward pass with ```call``` method. Enables [eager execution](https://www.tensorflow.org/guide/eager#build_a_model)

In [0]:
import numpy as np

features = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

In [0]:
# maximum simplicity
seq_model = Sequential()
seq_model.add(Dense(20, activation='relu', input_shape=(32,)))
seq_model.add(Dense(20, activation='relu'))
seq_model.add(Dense(10, activation='softmax'))

seq_model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

seq_model.fit(features, labels, epochs=10, batch_size=32)

In [0]:
# more full featured and flexible
inputs = keras.Input(shape=(32,))
x = Dense(20, activation='relu')(inputs)
x = Dense(20, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

func_model = keras.Model(inputs, outputs)
func_model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
func_model.fit(features, labels, epochs=10, batch_size=32)

In [0]:
# maximally flexible and hackable
class ScModel(keras.Model):
  
  def __init__(self):
    super(ScModel, self).__init__()
    self.dense1 = Dense(20, activation='relu')
    self.dense2 = Dense(20, activation='relu')
    self.dense3 = Dense(10, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    return self.dense3(x)
  
sc_model = ScModel()
sc_model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
sc_model.fit(features, labels, epochs=10, batch_size=32)

## Visualize model

The summary is useful for simple models, but can be confusing for models that have multiple inputs or outputs.

Keras also provides a function to create a plot of the network neural network graph that can make more complex models easier to understand.

The plot_model() function in Keras will create a plot of your network. This function takes a few useful arguments:

* ```model```: (required) The model that you wish to plot.
* ```to_file```: (required) The name of the file to which to save the plot.
* ```show_shapes```: (optional, defaults to False) Whether or not to show the output shapes of each layer.
* ```show_layer_names```: (optional, defaults to True) Whether or not to show the name for each layer.

In [0]:
seq_model.summary()

In [0]:
mod = seq_model

tf.keras.utils.plot_model(
    mod, 
    to_file='{}.png'.format(mod), 
    show_shapes=True, 
    show_layer_names=True
)

from IPython.display import Image
Image(retina=True, filename='{}.png'.format(mod))

### Evaluate and predict

The `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods can use NumPy
data and a `tf.data.Dataset`.

To *evaluate* the inference-mode loss and metrics for the data provided:

In [0]:
# Unseen data
new_data  = np.random.random((1000, 32))
new_labels = np.random.random((1000, 10))

In [0]:
results = sc_model.predict(new_data)
result[0]

In [0]:
sc_model.evaluate(new_data, new_labels)

### Input tf.data datasets

Use the Dataset API to scale to large datasets
or multi-device training. Pass a `tf.data.Dataset` instance to the `fit`
method:

Here, the `fit` method uses the `steps_per_epoch` argument—this is the number of
training steps the model runs before it moves to the next epoch. Since the
`Dataset` yields batches of data, this snippet does not require a `batch_size`.

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

In [0]:
sc_model.fit(dataset, epochs=10, steps_per_epoch=30)

### Custom layers

Create a custom layer by subclassing `tf.keras.layers.Layer` and implementing
the following methods:

* `build`: Create the weights of the layer. Add weights with the `add_weight`
  method.
* `call`: Define the forward pass.
* `compute_output_shape`: Specify how to compute the output shape of the layer
  given the input shape.
* Optionally, a layer can be serialized by implementing the `get_config` method
  and the `from_config` class method.

Here's an example of a custom layer that implements a `matmul` of an input with
a kernel matrix:

In [0]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    # Create a trainable weight variable for this layer.
    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
    # Make sure to call the `build` method at the end
    super(MyLayer, self).build(input_shape)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [0]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# The compile step specifies the training configuration
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(features, labels, batch_size=32, epochs=5)

## Callbacks

A callback is an object passed to a model to customize and extend its behavior
during training. You can write your own custom callback, or use the built-in
`tf.keras.callbacks` that include:

* `tf.keras.callbacks.ModelCheckpoint`: Save checkpoints of your model at
  regular intervals.
* `tf.keras.callbacks.LearningRateScheduler`: Dynamically change the learning
  rate.
* `tf.keras.callbacks.EarlyStopping`: Interrupt training when validation
  performance has stopped improving.
* `tf.keras.callbacks.TensorBoard`: Monitor the model's behavior using
  [TensorBoard](./summaries_and_tensorboard.md).

To use a `tf.keras.callbacks.Callback`, pass it to the model's `fit` method:

In [0]:
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(features, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(new_data, new_labels))

## Model Saving and Restoring
### Weights only

Save and load the weights of a model using `tf.keras.Model.save_weights`:

### Configuration only

A model's configuration can be saved—this serializes the model architecture
without any weights. A saved configuration can recreate and initialize the same
model, even without the code that defined the original model. Keras supports
JSON and YAML serialization formats:

### Entire model

The entire model can be saved to a file that contains the weight values, the
model's configuration, and even the optimizer's configuration. This allows you
to checkpoint a model and resume training later—from the exact same
state—without access to the original code.

In [1]:
# Create a trivial model
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(32,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(features, labels, batch_size=32, epochs=5)


# Save entire model to a HDF5 file
model.save('my_model.h5')

# Recreate the exact same model, including weights and optimizer.
model = tf.keras.models.load_model('my_model.h5')

NameError: ignored

## Pretrained Models

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import tensorflow.keras.backend as K
import numpy as np

In [3]:
model = ResNet50(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.
102858752/102853048 [==============================] - 1s 0us/step


[imagenet classes](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)

In [4]:
!wget -O test.png https://www.freepngimg.com/thumb/corn/23-corn-png-image-thumb.png
# !wget -O test.png https://vignette.wikia.nocookie.net/dino/images/f/f6/JW_triceratops.png/revision/latest?cb=20150407211112

--2019-02-18 15:48:43--  https://www.freepngimg.com/thumb/corn/23-corn-png-image-thumb.png
Resolving www.freepngimg.com (www.freepngimg.com)... 88.99.162.33
Connecting to www.freepngimg.com (www.freepngimg.com)|88.99.162.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38182 (37K) [image/png]
Saving to: ‘test.png’

test.png            100%[===================>]  37.29K  --.-KB/s    in 0.1s    

2019-02-18 15:48:43 (363 KB/s) - ‘test.png’ saved [38182/38182]



In [5]:
img_path = 'test.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

print('Predicted:', decode_predictions(preds, top=3)[0])

40960/35363 [==================================] - 0s 0us/step
Predicted: [('n12144580', 'corn', 0.64657545), ('n13133613', 'ear', 0.3533439), ('n02226429', 'grasshopper', 2.1798884e-05)]


In [10]:
# Get the embedding form the corn picture
get_last_layer_output = K.function(
    [model.layers[0].input],
    [model.layers[-1].output]
)

get_last_layer_output([x])[0].shape

(1, 1000)

In [9]:
# Other way to do the same thing, just create another model
temp_model = Model(
    [model.layers[0].input],
    [model.layers[-1].output]
)

temp_model.predict(x).shape

(1, 1000)

### Fine-tuning pre-trained model
Provided that our dataset is not drastically different in context to the original dataset, we can use pretrained models like the one above to fine tune to a different task

In [15]:
[print(l) for l in model.layers[-10:]]

[None, None, None, None, None, None, None, None, None, None]

In [16]:
# remove the softmax layer
model = Model(inputs=model.inputs, outputs=model.get_layer('avg_pool').output)
[print(l) for l in model.layers[-10:]]

[None, None, None, None, None, None, None, None, None, None]

In [21]:
# Freeze the previous layers
for layer in model.layers[:-1]:
    layer.trainable = False

# add a couple layers
x = Flatten()(model.layers[-1].output)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(500, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(100, activation='softmax')(x)

model = Model(inputs=model.input, outputs=out)

[print(l) for l in model.layers[-10:]]

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[None, None, None, None, None, None, None, None, None, None]

In [0]:
model.compile(optimizer=Adam(lr=0.001, momentum=0.9), loss='caregorical_crossentropy')
model.fit_generator(...,...)